# Bachelor Thesis Code - Lotte Michels - 2055481


In [ ]:
# Install/Import relevant modules

#!pip install mne # Uncomment this to install MNE-Python
import mne
from mne.preprocessing import ICA
import os
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Retrieve all the datafiles

datafiles = os.listdir('drive/MyDrive/Bachelorthesis/Raw Data') #
datafiles.sort()
print("There are {} files in total".format(len(datafiles)))
print(datafiles)

In [ ]:
# This code block will convert all datafiles to _raw.fif files

for p_number in range(1, 18): # The files for the first round of data collection (participants 1-17) are in a different format than the second round
  p_code = str(p_number)
  if p_number < 10:
    p_code = "0{}".format(p_code)

  LT_files = []
  for filename in datafiles:
    if p_code in filename and 'LT' in filename:
      raw = mne.io.read_raw_bdf("../Raw Data/{}".format(filename))
      LT_files.append(raw)
  if LT_files == []:
    print("No LT files found for EPM{}".format(p_code))
  else:
    LT = mne.concatenate_raws(LT_files)
    LT.save('drive/MyDrive/Bachelorthesis/Concatenated Files/EPM{}_LT_raw.fif'.format(p_code), overwrite=True)

  MT_files = []
  for filename in datafiles:
    if p_code in filename and 'MT' in filename:
      raw = mne.io.read_raw_bdf("../Raw Data/{}".format(filename))
      MT_files.append(raw)
  if MT_files == []:
    print("No MT files found for EPM{}".format(p_code))
  else:
    MT = mne.concatenate_raws(MT_files)
    MT.save('drive/MyDrive/Bachelorthesis/Concatenated Files/EPM{}_MT_raw.fif'.format(p_code), overwrite=True)

  DT_files = []
  for filename in datafiles:
    if p_code in filename and 'DT' in filename:
      raw = mne.io.read_raw_bdf("../Raw Data/{}".format(filename))
      DT_files.append(raw)
  if DT_files == []:
    print("No DT files found for EPM{}".format(p_code))
  else:
    DT = mne.concatenate_raws(DT_files)
    DT.save('drive/MyDrive/Bachelorthesis/Concatenated Files/EPM{}_DT_raw.fif'.format(p_code), overwrite=True)

  print("Completed EPM{}".format(p_code))
  print("")

for p_number in range(21, 33): # The files for the second round of data collection (participants 21-32) are in a different format than the first round
  p_code = str(p_number)

  LT_file = "EPM{}_LT.bdf".format(p_code)
  LT = mne.io.read_raw_bdf("../Second Round/{}".format(LT_file))
  LT.save('drive/MyDrive/Bachelorthesis/Concatenated Files/EPM{}_LT_raw.fif'.format(p_code), overwrite=True)

  if p_code == "24": #there are two MT files for p24 (technical issues during the experiment)
    MT_files = []
    f1 = mne.io.read_raw_bdf("../Second Round/EPM24_MT1.bdf")
    MT_files.append(f1)
    f2 = mne.io.read_raw_bdf("../Second Round/EPM24_MT2.bdf")
    MT_files.append(f2)
    MT = mne.concatenate_raws(MT_files)
    MT.save('drive/MyDrive/Bachelorthesis/Concatenated Files/EPM24_MT_raw.fif', overwrite=True)
  else:
    MT_file = "EPM{}_MT.bdf".format(p_code)
    MT = mne.io.read_raw_bdf("../Second Round/{}".format(MT_file))
    MT.save('drive/MyDrive/Bachelorthesis/Concatenated Files/EPM{}_MT_raw.fif'.format(p_code), overwrite=True)

  if p_code == "25": #there are two DT files for p25 (technical issues during the experiment)
    DT_files = []
    f1 = mne.io.read_raw_bdf("../Second Round/EPM25_DT1.bdf")
    DT_files.append(f1)
    f2 = mne.io.read_raw_bdf("../Second Round/EPM25_DT2.bdf")
    DT_files.append(f2)
    DT = mne.concatenate_raws(DT_files)
    DT.save('drive/MyDrive/Bachelorthesis/Concatenated Files/EPM25_DT_raw.fif', overwrite=True)
  else:
    DT_file = "EPM{}_DT.bdf".format(p_code)
    DT = mne.io.read_raw_bdf("../Second Round/{}".format(DT_file))
    DT.save('drive/MyDrive/Bachelorthesis/Concatenated Files/EPM{}_DT_raw.fif'.format(p_code), overwrite=True)

  print("Completed EPM{}".format(p_code))
  print("")

In [ ]:
# This code block will filter the data, run ICA and plot the ICA results for each file

for f in os.listdir("drive/MyDrive/Bachelorthesis/Concatenated Files"):
  filename = str(f)
  subject = filename.strip("EPM")
  subject = subject[0:2]
  if subject[0] == "0":
    subject = subject.strip("0")
  subject = int(subject)

  raw = mne.io.read_raw_fif("drive/MyDrive/Bachelorthesis/Concatenated Files/{}".format(filename), preload=True) #Or use testfile.load_data() for loading
  # To print file info use print(testfile.info)
  # To print the channels use print(testfile.ch_names)

  # Set reference
  raw.set_eeg_reference(ref_channels=['Cz'])

  # Adjust channel types
  if subject < 21: # the data collected for participants from the first round (1-17) were collected with a different system than in the second round
    raw.set_channel_types(mapping={'EXG1':'eog', 'EXG2':'eog', 'EXG3':'eog', 'EXG4':'eog', 'EXG5':'eog', 'EXG6':'eog', 'EXG7':'misc', 'EXG8':'misc', 'GSR1':'misc', 'GSR2':'misc', 'Erg1':'misc', 'Erg2':'misc', 'Resp':'misc', 'Plet':'misc', 'Temp':'misc', 'Status':'stim'})
    # Rereference to the mastoid links
    raw.set_eeg_reference(ref_channels=['EXG1', 'EXG2'])
  else: # the data collected for participants from the second round (21-32) were collected with a different system than in the first round
    raw.set_channel_types(mapping={'M1':'eog', 'M2':'eog', 'Up':'eog', 'Down':'eog', 'Left':'eog', 'Right':'eog', 'Status':'stim'}) 
    # Rereference to the mastoid links
    raw.set_eeg_reference(ref_channels=['M1', 'M2'])

  # Apply signal filtering (while maintaining the raw file)
  filt_raw = raw.copy()
  filt_raw.notch_filter(freqs=50) #to remove power artefacts from the measuring devices, which are known to be at 50 Hz
  filt_raw.filter(l_freq=0.1, h_freq=None)
  filt_raw.filter(l_freq=None, h_freq=100)

  # Align sensors with their correct position (EEG Sensor Cap)
  biosemi_montage = mne.channels.make_standard_montage('biosemi64')
  filt_raw.set_montage(biosemi_montage)
  #test.plot_sensors(show_names=True)

  # Independent Component Analysis (ICA)
  ica = ICA(n_components=20, max_iter='auto', random_state=97) # ICA fitting is not deterministic so we specify a random seed to get identical results each time we run the code
  ica.fit(filt_raw)

  # Plot ICA results
  print("ICA results for {}".format(filename))
  ica.plot_components()
  ica.plot_sources(filt_raw, show_scrollbars=False)


The plots created by the previous code block were visually inspected. 
Components to extract from the signal were manually selected and entered to the 'icas.txt' file that will be opened in the next code block.

In [ ]:
#Opening the ICAs textfile:
selected_icas = pd.read_csv('drive/MyDrive/Bachelorthesis/icas.txt', sep="; ", header=0, names=["Data", "ICAs"])
#pd.read_csv automatically converts the ICA lists to strings. To convert back:
selected_icas.loc[:,'ICAs'] = selected_icas.loc[:,'ICAs'].apply(lambda x: literal_eval(x))
#print(selected_icas)

In [ ]:
# Apply ICA

for f in os.listdir("drive/MyDrive/Bachelorthesis/Concatenated Files"):
  filename = str(f)
  subject = filename.strip("EPM")
  subject = subject[0:2]
  if subject[0] == "0":
    subject = subject.strip("0")
  raw = mne.io.read_raw_fif("drive/MyDrive/Bachelorthesis/Concatenated Files/{}".format(filename), preload=True) #Or use testfile.load_data() for loading
  # To print file info use print(testfile.info)
  # To print the channels use print(testfile.ch_names)

  # Set reference
  raw.set_eeg_reference(ref_channels=['Cz'])

  # Adjust channel types
  if subject < 21: # the data collected for participants from the first round (1-17) were collected with a different system than in the second round
    raw.set_channel_types(mapping={'EXG1':'eog', 'EXG2':'eog', 'EXG3':'eog', 'EXG4':'eog', 'EXG5':'eog', 'EXG6':'eog', 'EXG7':'misc', 'EXG8':'misc', 'GSR1':'misc', 'GSR2':'misc', 'Erg1':'misc', 'Erg2':'misc', 'Resp':'misc', 'Plet':'misc', 'Temp':'misc', 'Status':'stim'})
    # Rereference to the mastoid links
    raw.set_eeg_reference(ref_channels=['EXG1', 'EXG2'])
  else: # the data collected for participants from the second round (21-32) were collected with a different system than in the first round
    raw.set_channel_types(mapping={'M1':'eog', 'M2':'eog', 'Up':'eog', 'Down':'eog', 'Left':'eog', 'Right':'eog', 'Status':'stim'}) 
    # Rereference to the mastoid links
    raw.set_eeg_reference(ref_channels=['M1', 'M2'])

  # Apply signal filtering (while maintaining the raw file)
  filt_raw = raw.copy()
  filt_raw.notch_filter(freqs=50) #to remove power artefacts from the measuring devices, which are known to be at 50 Hz
  filt_raw.filter(l_freq=0.1, h_freq=None)
  filt_raw.filter(l_freq=None, h_freq=100)

  # Align sensors with their correct position (EEG Sensor Cap)
  biosemi_montage = mne.channels.make_standard_montage('biosemi64')
  filt_raw.set_montage(biosemi_montage)
  #test.plot_sensors(show_names=True)

  # Independent Component Analysis (ICA)
  ica = ICA(n_components=20, max_iter='auto', random_state=97) # ICA fitting is not deterministic so we specify a random seed to get identical results each time we run the code
  ica.fit(filt_raw)

  # Plot ICA results
  #print("ICA results for {}".format(filename))
  #ica.plot_components()
  #ica.plot_sources(filt_raw, show_scrollbars=False)

  # Pick ICA components
  s_filename = filename.strip("_raw.fif")
  ica_list = selected_icas.query("Data=='{}'".format(s_filename))["ICAs"].values.tolist()[0]
  ica.exclude = ica_list #We specify which ICAs to remove by setting the ica.exclude attribute
  reconst_raw = filt_raw.copy() 
  ica.apply(reconst_raw) #Here we actually apply (in place) the removal of the selected ICAs

  # Save filtered data
  path = "drive/MyDrive/Bachelorthesis/ICA Processed Files/{}_ica_raw.fif".format(s_filename) 
  reconst_raw.save(path, overwrite=True)

  print("COMPLETED {}".format(s_filename))


In [ ]:
# Define necessary variables for the epoching
meta_info = pd.read_csv('drive/MyDrive/Bachelorthesis/meta_info.csv', sep=";") #here we can see the accuracy per participant per trial
stimulus_info = pd.read_excel('drive/MyDrive/Bachelorthesis/stimuli.xlsx') #here we can see the sequence length per stimulus

fcc_event_dict = {'start FCC':101} # The FCC trials are indicated by the 101 markers
ecc_event_dict = {'start ECC':102} # The ECC trials are indicated by the 102 markers

r1_drop = ['EXG1', 'EXG2', 'EXG3', 'EXG4', 'EXG5', 'EXG6', 'EXG7', 'EXG8', 'GSR1', 'GSR2', 'Erg1', 'Erg2', 'Resp', 'Plet', 'Temp']
r2_drop = ['M1', 'M2', 'Up', 'Down', 'Left', 'Right']

In [ ]:
# Creating FCC Epochs

LT_FCC_Epochs = None
MT_FCC_Epochs = None
DT_FCC_Epochs = None

for f in os.listdir("drive/MyDrive/Bachelorthesis/ICA Processed Files"):
  filename = str(f)
  if filename == "EPM03_MT_ica_raw.fif" or filename == "EPM13_MT_ica_raw.fif" or filename == "EPM25_LT_ica_raw.fif": 
  #3_MT does not work (meta has 37 rows and there are 36 events for the ECC trials), 
  #13_MT does not work (meta has 36 rows and there are 35 events for both the FCC and ECC trials),
  #25_LT does not work (meta has 36 rows while there are only 22 events)
    continue
  print("\nWorking on {}".format(filename))
  task = filename.strip("_ica_raw.fif")[-2:]
  subject = filename.strip("{}_ica_raw.fif".format(task))
  subject = subject.strip("EPM")
  if subject[0] == "0":
    subject.strip("0")
  print(subject)

  # Load file and create epochs
  raw = mne.io.read_raw_fif("drive/MyDrive/Bachelorthesis/ICA Processed Files/{}".format(filename), preload=True) 
  if int(subject) < 21: #to make sure all epochs have the same number of channels
    raw.drop_channels(r1_drop)
  else:
    raw.drop_channels(r2_drop)
  events = mne.find_events(raw, stim_channel="Status") 
  fcc_epochs = mne.Epochs(raw, events, event_id=fcc_event_dict, tmin=-1, tmax=3.6, preload=True) #all fcc trials are 3.6 seconds long (6 items)

  # Add metadata
  fcc_metadata = {} # To link the subject number and response accuracy to the epochs. Later we turn this into a pd.DataFrame

  participantx = meta_info[meta_info["Subject"] == int(subject)]
  task_px = participantx[participantx["Session"] == task]
  if filename == "EPM04_MT_ica_raw.fif":  #4_MT is missing EEG data from block 5 (see log-book)
    for x in range(501, 537):
      task_px = task_px[task_px["BlockTrial"] != x]
  fcc_tpx = task_px[task_px["Condition"] == "FCC"]
  fcc_responses = list(fcc_tpx["Accuracy"]) 
  # Add the list of response accuracies to the metadata dictionary
  fcc_metadata["accuracy"] = fcc_responses
  # Add participantnumber to metadata dictionary
  subject_meta = [subject for _ in range(len(fcc_responses))]
  fcc_metadata["subject"] = subject_meta
  #print(metadata)
  fcc_meta_df = pd.DataFrame(fcc_metadata)
  fcc_epochs.metadata = fcc_meta_df
  #print(fcc_epochs.metadata)

  # Concatenate Epochs
  if task == "LT":
    if LT_FCC_Epochs == None:
      LT_FCC_Epochs = fcc_epochs
    else:
      LT_FCC_Epochs = mne.concatenate_epochs([LT_FCC_Epochs, fcc_epochs])
  if task == "MT":
    if MT_FCC_Epochs == None:
      MT_FCC_Epochs = fcc_epochs
    else:
      MT_FCC_Epochs = mne.concatenate_epochs([MT_FCC_Epochs, fcc_epochs])
  if task == "DT":
    if DT_FCC_Epochs == None:
      DT_FCC_Epochs = fcc_epochs
    else:
      DT_FCC_Epochs = mne.concatenate_epochs([DT_FCC_Epochs, fcc_epochs])

LT_FCC_Epochs.save('drive/MyDrive/Bachelorthesis/Epochs/LT_FCC_Epochs.fif', overwrite=True)
MT_FCC_Epochs.save('drive/MyDrive/Bachelorthesis/Epochs/MT_FCC_Epochs.fif', overwrite=True)
DT_FCC_Epochs.save('drive/MyDrive/Bachelorthesis/Epochs/DT_FCC_Epochs.fif', overwrite=True)


In [ ]:
# Creating ECC Epochs

LT_ECC9_Epochs = None
LT_ECC10_Epochs = None
LT_ECC11_Epochs = None
LT_ECC12_Epochs = None

MT_ECC9_Epochs = None
MT_ECC10_Epochs = None
MT_ECC11_Epochs = None
MT_ECC12_Epochs = None

DT_ECC9_Epochs = None
DT_ECC10_Epochs = None
DT_ECC11_Epochs = None
DT_ECC12_Epochs = None

for f in os.listdir("drive/MyDrive/Bachelorthesis/ICA Processed Files"):
  filename = str(f)
  if filename == "EPM03_MT_ica_raw.fif" or filename == "EPM13_MT_ica_raw.fif" or filename == "EPM25_LT_ica_raw.fif": 
  #3_MT does not work (meta has 37 rows and there are 36 events for the ECC trials), 
  #13_MT does not work (meta has 36 rows and there are 35 events for both the FCC and ECC trials),
  #25_LT does not work (meta has 36 rows while there are only 22 events)
    continue
  print("\n Working on {}".format(filename))
  task = filename.strip("_ica_raw.fif")[-2:]
  subject = filename.strip("{}_ica_raw.fif".format(task))
  subject = subject.strip("EPM")
  if subject[0] == "0":
    subject.strip("0")

  # Load file and create epochs
  raw = mne.io.read_raw_fif("drive/MyDrive/Bachelorthesis/ICA Processed Files/{}".format(filename), preload=True) 
  if int(subject) < 21: #to make sure all epochs have the same number of channels
    raw.drop_channels(r1_drop)
  else:
    raw.drop_channels(r2_drop)
  events = mne.find_events(raw, stim_channel="Status") 
  ecc_epochs = mne.Epochs(raw, events, event_id=ecc_event_dict, tmin=-1, tmax=7.2, preload=True) # 7.2 seconds is the longest ECC trial

  # Add metadata
  ecc_metadata = {} # To link the participant number, sequence length and response accuracy to the epochs. Later we turn this into a pd.DataFrame 

  participantx = meta_info[meta_info["Subject"] == int(subject)]
  task_px = participantx[participantx["Session"] == task]
  if filename == "EPM04_MT_ica_raw.fif":  #4_MT is missing EEG data from block 5 (see log-book)
    for x in range(501, 537):
      task_px = task_px[task_px["BlockTrial"] != x]
  ecc_tpx = task_px[task_px["Condition"] == "ECC"]
  ecc_responses = list(ecc_tpx["Accuracy"]) 
  # Add the list of response accuracies to the metadata dictionary
  ecc_metadata["accuracy"] = ecc_responses
  # Add the subject number to the metadata dictionary
  subject_meta = [subject for _ in range(len(ecc_responses))]
  ecc_metadata["subject"] = subject_meta
  ecc_stimulus_length = []
  for stimulus in ecc_tpx["audio"]:
    for soundname in stimulus_info["SoundStimNRWords"]:
      if stimulus in soundname:
        # Add the length of the stimulus to the list
        ecc_stimulus_length.append(list(stimulus_info[stimulus_info["SoundStimNRWords"]==soundname]["NRWords"])[0])
        break
  # Add the list of stimulus lengths to the metadata dictionary        
  ecc_metadata["stim_length"] = ecc_stimulus_length
  #print(metadata)
  ecc_meta_df = pd.DataFrame(ecc_metadata)
  ecc_epochs.metadata = ecc_meta_df
  #print(ecc_epochs.metadata)

  # Crop epochs based on sequence length
  ecc9 = ecc_epochs['stim_length == 9'].crop(tmin=-1, tmax=5.4)
  ecc10 = ecc_epochs['stim_length == 10'].crop(tmin=-1, tmax=6.0)
  ecc11 = ecc_epochs['stim_length == 11'].crop(tmin=-1, tmax=6.6)
  ecc12 = ecc_epochs['stim_length == 12']

  # Concatenate Epochs
  if task == "LT":
    if LT_ECC9_Epochs == None:
      LT_ECC9_Epochs = ecc9
      LT_ECC10_Epochs = ecc10
      LT_ECC11_Epochs = ecc11
      LT_ECC12_Epochs = ecc12
    else:
      LT_ECC9_Epochs = mne.concatenate_epochs([LT_ECC9_Epochs, ecc9])
      LT_ECC10_Epochs = mne.concatenate_epochs([LT_ECC10_Epochs, ecc10])
      LT_ECC11_Epochs = mne.concatenate_epochs([LT_ECC11_Epochs, ecc11])
      LT_ECC12_Epochs = mne.concatenate_epochs([LT_ECC12_Epochs, ecc12])
  if task == "MT":
    if MT_ECC9_Epochs == None:
      MT_ECC9_Epochs = ecc9
      MT_ECC10_Epochs = ecc10
      MT_ECC11_Epochs = ecc11
      MT_ECC12_Epochs = ecc12
    else:
      MT_ECC9_Epochs = mne.concatenate_epochs([MT_ECC9_Epochs, ecc9])
      MT_ECC10_Epochs = mne.concatenate_epochs([MT_ECC10_Epochs, ecc10])
      MT_ECC11_Epochs = mne.concatenate_epochs([MT_ECC11_Epochs, ecc11])
      MT_ECC12_Epochs = mne.concatenate_epochs([MT_ECC12_Epochs, ecc12])
  if task == "DT":
    if DT_ECC9_Epochs == None:
      DT_ECC9_Epochs = ecc9
      DT_ECC10_Epochs = ecc10
      DT_ECC11_Epochs = ecc11
      DT_ECC12_Epochs = ecc12
    else:
      DT_ECC9_Epochs = mne.concatenate_epochs([DT_ECC9_Epochs, ecc9])
      DT_ECC10_Epochs = mne.concatenate_epochs([DT_ECC10_Epochs, ecc10])
      DT_ECC11_Epochs = mne.concatenate_epochs([DT_ECC11_Epochs, ecc11])
      DT_ECC12_Epochs = mne.concatenate_epochs([DT_ECC12_Epochs, ecc12])

LT_ECC9_Epochs.save('drive/MyDrive/Bachelorthesis/Epochs/LT_ECC9_Epochs.fif', overwrite=True)
LT_ECC10_Epochs.save('drive/MyDrive/Bachelorthesis/Epochs/LT_ECC10_Epochs.fif', overwrite=True)
LT_ECC11_Epochs.save('drive/MyDrive/Bachelorthesis/Epochs/LT_ECC11_Epochs.fif', overwrite=True)
LT_ECC12_Epochs.save('drive/MyDrive/Bachelorthesis/Epochs/LT_ECC12_Epochs.fif', overwrite=True)

MT_ECC9_Epochs.save('drive/MyDrive/Bachelorthesis/Epochs/MT_ECC9_Epochs.fif', overwrite=True)
MT_ECC10_Epochs.save('drive/MyDrive/Bachelorthesis/Epochs/MT_ECC10_Epochs.fif', overwrite=True)
MT_ECC11_Epochs.save('drive/MyDrive/Bachelorthesis/Epochs/MT_ECC11_Epochs.fif', overwrite=True)
MT_ECC12_Epochs.save('drive/MyDrive/Bachelorthesis/Epochs/MT_ECC12_Epochs.fif', overwrite=True)

DT_ECC9_Epochs.save('drive/MyDrive/Bachelorthesis/Epochs/DT_ECC9_Epochs.fif', overwrite=True)
DT_ECC10_Epochs.save('drive/MyDrive/Bachelorthesis/Epochs/DT_ECC10_Epochs.fif', overwrite=True)
DT_ECC11_Epochs.save('drive/MyDrive/Bachelorthesis/Epochs/DT_ECC11_Epochs.fif', overwrite=True)
DT_ECC12_Epochs.save('drive/MyDrive/Bachelorthesis/Epochs/DT_ECC12_Epochs.fif', overwrite=True)

In [ ]:
# Results will be formatted as a table (.csv file) with columns:
  # - subject  
  # - task (LT/MT/DT)
  # - condition (ECC/FCC)
  # - electrode (64 in total)
  # - alpha power (average over the band frequencies and over corresponding trials)
  # - response accuracy (average over corresponding trials)
  # - sequence length (6 for FCC, 9, 10, 11 or 12 for ECC)

In [ ]:
results = { # to store the results; for now include the 'epoch' and later average the alpha_power over corresponding epochs
    "subject":[],
    "task":[],
    "condition":[],
    "epoch":[],
    "electrode":[],
    "alpha_power":[],
    "response_acc":[],
    "stimulus_len":[]
} 

for f in os.listdir("drive/MyDrive/Bachelorthesis/Epochs"):
  filename = str(f)
  print("\nWorking on {}".format(filename))
  task = filename[0:2]
  if "ECC" in filename:
    condition = "ECC"
    if "9" in filename:
      stim_len = 9
    elif "10" in filename:
      stim_len = 10
    elif "11" in filename:
      stim_len = 11
    elif "12" in filename:
      stim_len = 12
  else:
    condition = "FCC"
    stim_len = 6
  # Load epochs
  epoch_obj = mne.read_epochs("drive/MyDrive/Bachelorthesis/Epochs/{}".format(filename), preload=True)

  # Compute TFR power rates
  freqs = [8, 9, 10, 11, 12]
  n_cycles = [4, 4.5, 5, 5.5, 6]
  power = mne.time_frequency.tfr_morlet(epoch_obj, average=False, return_itc=False, n_cycles=n_cycles, freqs=freqs) # average=False so that TFR is computed for each epoch seperately
  #this function creates a EpochsTFR container with among others the attributes:
  # - data (4D-array with power values; dims are n_epochs, n_channels, n_freqs and time)
  # - ch_names 
  # - metadata

  # Loop over the results and update results dictionary
  for epoch_nr in range(0, power.data.shape[0]):
    epoch = epoch_nr+1
    response = list(power.metadata["accuracy"])[epoch_nr]
    subject = list(power.metadata["subject"])[epoch_nr]
    for chan_nr in range(0, power.data.shape[1]):
      #subject
      results["subject"].append(subject)
      #task
      results["task"].append(task)
      #condition
      results["condition"].append(condition)
      #epoch
      results["epoch"].append(epoch)
      #channel
      results["electrode"].append(power.ch_names[chan_nr])
      #alpha_power
      results["alpha_power"].append(np.average(power.data[epoch_nr, chan_nr]))
      #response_acc
      results["response_acc"].append(response)
      #stimulus_len
      results["stimulus_len"].append(stim_len)

  # To compute average power over all epochs:
  #power.average()

In [ ]:
# Average results over corresponding epochs

final_results = {
    "subject":[],
    "task":[],
    "condition":[],
    "electrode":[],
    "alpha_power":[], #averaged over corresponding trials
    "response_acc":[], #averaged over corresponding trials
    "stimulus_len":[]
}

#subjects = list(set(results["subject"])).sort()
tasks = ["MT", "LT", "DT"]
conditions = ["ECC", "FCC"]
subjects = list(set(results["subject"]))
subjects.sort()
electrodes = list(set(results["electrode"]))
electrodes.sort()
lengths = [9, 10, 11, 12]

# Subset the results dataframe and compute average power and response values.
# Add results to the new 'final_results' dictionary
for s in subjects:
  print("Working on subject:", s)
  for t in tasks:
    for c in conditions:
      for e in electrodes:
        if c == "ECC":
          for l in lengths:
            subdf = results[results["subject"] == s][results["task"] == t][results["condition"] == c][results["electrode"] == e][results["stimulus_len"] == l]
            av_pow = subdf["alpha_power"].mean()
            av_res = subdf["response_acc"].mean()
            #subject
            final_results["subject"].append(s)
            #task
            final_results["task"].append(t)
            #condition
            final_results["condition"].append(c)
            #channel
            final_results["electrode"].append(e)
            #alpha_power
            final_results["alpha_power"].append(av_pow)
            #response_acc
            final_results["response_acc"].append(av_res)
            #stimulus_len
            final_results["stimulus_len"].append(l)
        else:
          subdf = results[results["subject"] == s][results["task"] == t][results["condition"] == c][results["electrode"] == e]
          av_pow = subdf["alpha_power"].mean()
          av_res = subdf["response_acc"].mean()
          #subject
          final_results["subject"].append(s)
          #task
          final_results["task"].append(t)
          #condition
          final_results["condition"].append(c)
          #channel
          final_results["electrode"].append(e)
          #alpha_power
          final_results["alpha_power"].append(av_pow)
          #response_acc
          final_results["response_acc"].append(av_res)
          #stimulus_len
          final_results["stimulus_len"].append(6)


Final file should look like:
- Subjects (25 participants)
- Task (3 levels: MT, LT, DT)
- Condition (2 levels: ECC, FCC)
- Hemisphere (2: left, right)
- Region (3 levels: frontal, central, posterior)
- Power 
- Response accuracy
- Stimulus length

In [ ]:
# Define brain regions
left = ["Fp1", "AF7", "AF3", "F7", "F5", "F3", "F1", "FT7", "FC5", "FC3", "FC1", "T7", "C5", "C3", "C1", "TP7", "CP5", "CP3", "CP1", "P9", "P7", "P5", "P3", "P1", "PO7", "PO3", "O1"]
right = ["FP2", "AF4", "AF8", "F2", "F4", "F6", "F8", "FC2", "FC4", "FC6", "FT8", "C2", "C4", "C6", "T8", "CP2", "CP4", "CP6", "TP8", "P2", "P4", "P6", "P8", "P10", "PO4", "PO8", "O2"]
frontal = ["Fp1", "AF7", "AF3", "F7", "F5", "F3", "F1", "FT7", "FC5", "FC3", "FC1", "FP2", "AF4", "AF8", "F2", "F4", "F6", "F8", "FC2", "FC4", "FC6", "FT8", "Fpz", "AFz", "Fz", "FCz"]
central = ["T7", "C5", "C3", "C1", "TP7", "CP5", "CP3", "CP1", "C2", "C4", "C6", "T8", "CP2", "CP4", "CP6", "TP8", "Cz", "CPz"]
posterior = ["P9", "P7", "P5", "P3", "P1", "PO7", "PO3", "O1", "P2", "P4", "P6", "P8", "P10", "PO4", "PO8", "O2", "Pz", "POz", "Oz", "Iz"]

In [ ]:
# Reformat results by averaging over brain regions
results = final_results

final_results = { # to store the results
#per subject/task/condition/stim_len: group the electrodes into regions and hemispheres and compute the average alpha power
    "subject":[],
    "task":[],
    "condition":[],
    "hemisphere":[],
    "region":[],
    "alpha_power":[],
    "response_acc":[],
    "stimulus_len":[]
} 

subjects = list(set(results["subject"]))
subjects.sort()
tasks = ["MT", "LT", "DT"]
conditions = ["ECC", "FCC"]
hemispheres = [left, right]
regions = [frontal, central, posterior]
electrodes = list(set(results["electrode"]))
electrodes.sort()
lengths = [9, 10, 11, 12]

# Subset the final_results dataframe and compute average power and response values.
# Add results to the new 'final' dictionary
for s in subjects:
  print("Working on subject:", s)
  for t in tasks:
    for c in conditions: 
      for h in hemispheres:
        for r in regions:
          if c == "ECC":
            for l in lengths:
              subdf = results[results["subject"] == s][results["task"] == t][results["condition"] == c][results["stimulus_len"] == l]
              subdf = subdf[subdf["electrode"].isin(h)][subdf["electrode"].isin(r)]
              av_pow = subdf["alpha_power"].mean()
              av_res = subdf["response_acc"].mean()
              #subject
              final_results["subject"].append(s)
              #task
              final_results["task"].append(t)
              #condition
              final_results["condition"].append(c)
              #hemisphere
              if h == left:
                final_results["hemisphere"].append("left")
              else:
                final_results["hemisphere"].append("right")
              #region
              if r == frontal:
                final_results["region"].append("frontal")
              elif r == central:
                final_results["region"].append("central")
              else:
                final_results["region"].append("posterior")
              #alpha_power
              final_results["alpha_power"].append(av_pow)
              #response_acc
              final_results["response_acc"].append(av_res)
              #stimulus_len
              final_results["stimulus_len"].append(l)
          else:
            subdf = results[results["subject"] == s][results["task"] == t][results["condition"] == c]
            subdf = subdf[subdf["electrode"].isin(h)][subdf["electrode"].isin(r)]
            av_pow = subdf["alpha_power"].mean()
            av_res = subdf["response_acc"].mean()
            #subject
            final_results["subject"].append(s)
            #task
            final_results["task"].append(t)
            #condition
            final_results["condition"].append(c)
            #hemisphere
            if h == left:
              final_results["hemisphere"].append("left")
            else:
              final_results["hemisphere"].append("right")
            #region
            if r == frontal:
              final_results["region"].append("frontal")
            elif r == central:
              final_results["region"].append("central")
            else:
              final_results["region"].append("posterior")              
            #alpha_power
            final_results["alpha_power"].append(av_pow)
            #response_acc
            final_results["response_acc"].append(av_res)
            #stimulus_len
            final_results["stimulus_len"].append(6)

In [ ]:
df = pd.DataFrame.from_dict(final_results)
#print(df)
# Export final results
df.to_csv("drive/MyDrive/Bachelorthesis/results by region.csv")
df.to_excel("drive/MyDrive/Bachelorthesis/results by region.xlsx")